# Growth Rate Computation of MPAS Experiments

## import package

In [3]:
import sys
import numpy as np
import joblib as jl

sys.path.append('/home/b11209013/Package/')
import Theory as th # type: ignore
import DataProcess as dp # type: ignore
import SignalProcess as sp # type: ignore

## Load data

In [10]:
# case name
case: str = 'CNTL'

# path
fname: str = '/home/b11209013/2024_Research/MPAS/PCA_file/CNTL_PC.joblib'

# load data
data: dict = jl.load(fname)

# extract data
pc: dict[str: np.ndarray] = {}

lon : np.ndarray = data['lon']
lat : np.ndarray = data['lat']
time: np.ndarray = data['time']

pc['tpc1']  = data['pc1']['t']
pc['tpc2']  = data['pc2']['t']
pc['q1pc1'] = data['pc1']['q1']
pc['q1pc2'] = data['pc2']['q1']